<a href="https://colab.research.google.com/github/DrDjango/MarketingProjects_AI_ML/blob/main/Review_Translation%26Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-cloud-translate==2.0.1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from google.cloud import translate_v2
import os

In [ ]:
# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service-account.json"

In [ ]:
def translate_text(target: str, text: str) -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language="en")

    print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))

    return result

In [ ]:
from google.cloud import translate_v2 as translate

translate_text("en", "Recebi bem antes do prazo estipulado")

Text: Recebi bem antes do prazo estipulado
Translation: I received it well before the stipulated deadline
Detected source language: pt


{'translatedText': 'I received it well before the stipulated deadline',
 'detectedSourceLanguage': 'pt',
 'input': 'Recebi bem antes do prazo estipulado'}

In [ ]:
import pandas as pd

In [ ]:
from google.cloud import translate_v2 as translate
import pandas as pd
import os

translate_client = translate.Client()

def translate_text(target, text):
    """Translates text into the target language.

    Args:
        target (str): The language to translate the text into.
        text (str): The text to translate.

    Returns:
        str: The translated text.
    """
    if text:
        result = translate_client.translate(text, target_language=target)
        return result['translatedText']
    return text

def translate_reviews(input_file, output_file, target_language='en'):
    """Translates review comments in a CSV file and saves the results to another CSV file.

    Args:
        input_file (str): The path to the input CSV file.
        output_file (str): The path to the output CSV file.
        target_language (str, optional): The target language for translation. Defaults to 'en'.
    """
    # Load the reviews
    data = pd.read_csv(input_file)

    # Translate each review comment
    data['review_comment_message_translated'] = data['review_comment_message'].apply(lambda x: translate_text(target_language, x) if pd.notnull(x) else x)

    # Save the result
    data.to_csv(output_file, index=False)

# Define your input and output file paths
input_file_path = 'C:/Users/ankit/OneDrive - St. Lawrence College/Winter 2024/Artificial Intelligence in Marketing/Assignment 5/archive(11)/olist_order_reviews_dataset_clean.csv'
output_file_path = 'output_translated.csv'

# Perform the translation
translate_reviews(input_file_path, output_file_path, 'en')

print("Translation completed and saved to:", output_file_path)


Translation completed and saved to: output_translated.csv


In [ ]:
from google.cloud import language_v1

In [ ]:
def analyze_sentiment(text_content):
    """Analyzes sentiment in a given text."""
    client = language_v1.LanguageServiceClient()

    # Prepare the text content
    document = language_v1.Document(content=text_content, type_=language_v1.Document.Type.PLAIN_TEXT)

    # Detect sentiment in the document
    sentiment = client.analyze_sentiment(request={"document": document}).document_sentiment

    return sentiment.score, sentiment.magnitude

In [ ]:
def perform_sentiment_analysis(input_file, output_file):
    # Load the translated reviews
    data = pd.read_csv(input_file)

    # Perform sentiment analysis
    sentiments = data['review_comment_message_translated'].apply(
        lambda x: analyze_sentiment(x) if pd.notnull(x) else (None, None)
    )

    # Split the sentiment scores and magnitudes into separate columns
    data['sentiment_score'], data['sentiment_magnitude'] = zip(*sentiments)

    # Save the result
    data.to_csv(output_file, index=False)
    # Define your input and output file paths
input_file_path = 'output_translated.csv'
output_file_path = 'olist_order_reviews_dataset_with_sentiment.csv'

In [ ]:
# Perform sentiment analysis
perform_sentiment_analysis(input_file_path, output_file_path)

print(f"Sentiment analysis completed and saved to:", output_file_path)

Sentiment analysis completed and saved to: olist_order_reviews_dataset_with_sentiment.csv


In [ ]:
!pip install pyodbc
!pip install pypyodbc

In [ ]:
import pyodbc

In [ ]:
# Load the dataset
data_path = 'olist_order_reviews_dataset_with_sentiment.csv'
data = pd.read_csv(data_path)

In [ ]:
pyodbc.drivers()

['SQL Server',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [ ]:
conn = pyodbc.connect(
    Trusted_Connection='yes',
    Driver='{ODBC Driver 17 for SQL Server}',
    Server='ANKIT\SQLEXPRESS',
    Database='Sentiment'
)

cursor = conn.cursor()

In [ ]:
print(conn)

In [ ]:
data.head

<bound method NDFrame.head of                             review_id                          order_id  \
0    8670d52e15e00043ae7de4c01cc2fe06  b9bf720beb4ab3728760088589c62129   
1    3948b09f7c818e2d86c9a546758b2335  e51478e7e277a83743b6f9991dbfa3fb   
2    373cbeecea8286a2b66c97b1b157ec46  583174fbe37d3d5f0d6661be3aad1786   
3    d21bbc789670eab777d27372ab9094cc  4fc44d78867142c627497b60a7e0228a   
4    c92cdd7dd544a01aa35137f901669cdf  37e7875cdce5a9e5b3a692971f370151   
..                                ...                               ...   
194  a44891f74fe94f29dbbfa6f7b60cceba  18c1040bac4b6a31a10b6d4201072ce2   
195  89411999cf6c8324d7e03f39b5eb6e9d  3c91fb2601d8d06e7e3c9ee4f79060c0   
196  712d37ca4d8dd3ac08c756c646b512cb  363814e98fb75811e4465c3b7fc84ca5   
197  4d3ff9f8c57cc584747eb184ef4cdebe  c54d822dcfd0f7c672574a607c9403b3   
198  cde63556ef00a605bff898ceb2f4d559  f099a4bf3c8c87b391cbb899f02d8e47   

     review_score     review_comment_title  \
0               4      

In [ ]:
data.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'review_comment_message_translated',
       'sentiment_score', 'sentiment_magnitude'],
      dtype='object')

In [ ]:
import pyodbc

# Establish a connection to the SQL Server database
conn = pyodbc.connect(
    Trusted_Connection='yes',
    Driver='{ODBC Driver 17 for SQL Server}',
    Server='ANKIT\SQLEXPRESS',
    Database='Sentiment'
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute the CREATE TABLE query
cursor.execute('CREATE TABLE Reviews(review_comment_message_translated NVARCHAR(MAX), sentiment_score FLOAT, sentiment_magnitude FLOAT)')

In [ ]:
for row in data.itertuples():
    cursor.execute('''
        INSERT INTO Reviews (review_comment_message_translated, sentiment_score, sentiment_magnitude)
        VALUES (?,?,?)
    ''',
    row.review_comment_message_translated,
    row.sentiment_score,
    row.sentiment_magnitude
    )

    conn.commit()